Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [106]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
        
import choice

from copy import deepcopy
from copy import copy
from dataclasses import dataclass
from typing import Callable
from matplotlib import pyplot as plt
from matplotlib import cm
from tqdm.notebook import tqdm


## The *Nim* and *Nimply* classes

In [107]:
Nimply = namedtuple("Nimply", "row, num_objects")



In [108]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)
    

    @rows.setter
    def rows(self, new_rows):
            self._rows = list(new_rows)


    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [109]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [110]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    print(possible_moves)
    print(Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1]))))
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [111]:
@dataclass
class Individual:
    fitness: tuple
    move: Nimply

def fitness2(state: Nim,move :Nimply):  ## (#numerorighe>1,numerorighe>0,xor) 

    tmp = deepcopy(state)
    tmp.nimming(ply=move)
    xor=nim_sum(tmp)
    num_active_rows = sum(o > 0 for o in state.rows)
    num_rows_maj1 = sum(o > 1 for o in state.rows)
    
    return num_rows_maj1,num_active_rows, xor


def fitnessx(state: Nim,move :Nimply):  ## (#numerorighe>1,numerorighe>0,xor) 

    tmp = deepcopy(state)
    tmp.nimming(ply=move)
    xor=nim_sum(tmp)
    num_active_rows = sum(o > 0 for o in state.rows)
    num_rows_maj1 = sum(o > 1 for o in state.rows)
    
     # (#n_row>1,#n_row>0,xor)       otherwise level 50
     # (x,#indifferent,xor=0) best   level x     
    if(num_rows_maj1>1 and xor==0):
        return num_rows_maj1
   
     # (1, #odd,indifferent) best    level 1
    if(num_rows_maj1==1 and (num_active_rows%2)==1):
        return 1
     # (0,indifferent,indifferent) just win  level 0
    if(num_rows_maj1==0):
        return 0
   #da aggiustare level 1 perchè deve dare sempre dispari -> levo il lvl 1 e metto solo lo 0 con #row>0 dispari
    return 50


def fitness2(state: Nim,move :Nimply):  ## (#numerorighe>1,numerorighe>0,xor) 

    tmp = deepcopy(state)
    tmp.nimming(ply=move)
    xor=nim_sum(tmp)
    num_active_rows = sum(o > 0 for o in state.rows)
    num_rows_maj1 = sum(o > 1 for o in state.rows)
    
    return num_rows_maj1,num_active_rows, xor


def fitness(state: Nim,move :Nimply):  ## (#numerorighe>1,numerorighe>0,xor) 

    tmp = deepcopy(state)
    tmp.nimming(ply=move)
    xor=nim_sum(tmp)
    num_active_rows = sum(o > 0 for o in state.rows)
    num_rows_maj1 = sum(o > 1 for o in state.rows)
    
     # (#n_row>1,#n_row>0,xor)       otherwise level 50        midgame( xor=0)
     # (x,#indifferent,xor=0) best   level x                   endgame
    if(num_rows_maj1>1 and xor==0):                         # (2,1,1) ->  (1,1,1)
                                                            # (2,1,1,1)->(0,1,1,1)
        return num_rows_maj1                                                      
    
     # (0,#odd,indifferent) just win  level 0
    if(num_rows_maj1==0 and (num_active_rows%2)==1):
        return 0
   #da aggiustare level 1 perchè deve dare sempre dispari -> levo il lvl 1 e metto solo lo 0 con #row>0 dispari
    return 50




def retrieve_top(list_individual,mu):
  #sorted_list=list_individual.sort(key=scorefitness)
  sorted_list = sorted(list_individual, key=lambda x: x.fitness)

  
  return sorted_list[:mu]




def mutate2(ind: Individual,state: Nim) -> Individual:
    offspring = copy(ind)
    
    row, _ = offspring.move
      
    
    max_value=state.rows[row]
    new_num_object=random.randint(1,max_value)
    new_move=Nimply(row,new_num_object)
    tmp=Individual(fitness(state,new_move),new_move)
    
    return tmp



def mutate(ind: Individual,state: Nim) -> Individual:
    offspring = copy(ind)
    
      
    new_move=Nimply(offspring.move.row,offspring.move.num_objects-1)
    tmp=Individual(fitness(state,new_move),new_move)
    
    return tmp


def mutate_taboo(ind: Individual,state: Nim) -> Individual:
    offspring = copy(ind)
    new_value=offspring.move.num_objects-1;      
   # print("mutate")
   # print(new_value)
   # index=ind.move.row                                                        #newvalue= 3 ne prendo 3
  #  max_value=state.rows[index]
    while new_value >=1:
        new_move = Nimply(offspring.move.row, new_value)
        new_Individual = Individual(fitness(state, new_move), new_move)

        if new_Individual not in taboo:
            taboo.append(new_Individual)
            return new_Individual
        
        new_value -=1
    return None

def scorefitness(f1 ,f2):
      # (#numerorighe>1,numerorighe>0,xor)       
     # (x,#indiffernte,xor=0) best       
     # (1, #odd,indifferent) best
     # (0,indifferent,indifferent) just win
    if(f1[0]>1  and f2[0]>1):   #so the number_rows>1   are greather than 1
        if(f1[2]==0 and f2[2]!=0):
            return 1
        if(f1[2]!=0 and f2[2]==0):
            return 0
        if(f1[2]==0 and f2[2]==0 and f1[0]<f2[2]):
            return 1
        else:
            return 0
    if(f1[0]==1 and (f1[1]%2)==1): #numer_of_rows>1 is equals to 1 so we need to leave the state in an odd position like(2,0,0,0,0,0)
        return 1
    if(f2[0]==1 and (f2[1]%2)==1):
        return 0
     
    return 1

In [112]:
def adaptive(state: Nim) -> Nimply:
   taboo=[]
     
   mu=20
   lambdA=10
   #children_size=lambdA/mu
   children_size=20
   population_numply=  [Nimply(r, c) for r, c in enumerate(state.rows) if c];
   population_Individual= [  Individual(fitness(state,value),value) for value in population_numply]
   #population = random.sample(population_Individual, 30)
   population=population_Individual
        
   print(population)
   top_individual=None
   for iteration in range(5): #100 iteration
        
        for individual  in population:
            #fitness already calculated
            if (top_individual==None or  (top_individual.fitness>individual.fitness)):
                top_individual=individual
        #best_individual=retrieve_top(population,9)
        best_individual=population
        population= deepcopy(best_individual)
        
        for new_individual in best_individual:
            for ci in range(10):
                    
                    mutated_individual = mutate_taboo(new_individual, state)
                    if mutated_individual is not None:
                        population.append(mutated_individual)
        print(population)
               
   
      
  

   return top_individual.move
  
            



  




 #xor coppie si annullano :)       fitness 
 #numeri pari filati strategie 1 /numeri totali  less
 #numeri dispari  filati  strategia2 /numeri totali  less
 #numeri di fila stat 3 /numeri totali  less 
 #xor =6 levo il 6 se esiste :) 


In [113]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}") 
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    #best_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 0]
    if not spicy_moves:
       spicy_moves = list(analysis["possible_moves"].keys())
    #if not best_moves:
    #    best_moves = list(analysis["possible_moves"].keys())
   # if (best_moves):
    ply=random.choice(spicy_moves)
  #  else:
  #      ply = random.choice(spicy_moves)
    return ply


def very_optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    #logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = None
    count_1=0
    index_row=-1
    row_value=0
    
    
    for index,value in enumerate(state.rows):
     if(value)>1:                                                               
         count_1+=1                                                             
         index_row=index                                                           #   (X,#indiffernte,xor=0)
         row_value=value          # (#numerorighe>1,numerorighe>0,xor)                (1, #dispari,indifferente)
                                                                                      
                               #(0,xor=-1) vincente      (3,-5)  (4,-5) best   
    if(count_1>1):   # (1,xor=0) (2, xor=0) # (1,xor=1)    vincente      valori maggiori di uno
                     
        spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 0]
        ply=random.choice(spicy_moves)
        return ply
    if(count_1==1):# only 1 row with more than 1 object    ->   the goals is to keep odd active rows
     
        num_active_rows = sum(o > 0 for o in state.rows)
       # print(num_active_rows)
        #print(state.rows)
       
        if(num_active_rows%2!=0): #if even take all the object in that row
           # print(fitness(state,Nimply(index_row,row_value-1)))
            return Nimply(index_row,row_value-1)
        else:#leave only 1 object in that row
    
          return  Nimply(index_row,row_value)


   
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
     
    if not spicy_moves:
       spicy_moves = list(analysis["possible_moves"].keys())
   
    ply = random.choice(spicy_moves)
    return ply


##  K VERSION

In [114]:
import numpy as np

#K ---------Version
def gabriele_k(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1,min(c+1,state._k+1))]
    
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


def pure_random_k(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if (c > 0)])
    num_objects = random.randint(1, min(state.rows[row],state._k))
    return Nimply(row, num_objects)

def analize_k(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1,min(c+1,raw._k+1))):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal_k(state: Nim) -> Nimply:
    analysis = analize_k(state)
    logging.debug(f"analysis:\n{pformat(analysis)}") 
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    #best_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 0]
    if not spicy_moves:
       spicy_moves = list(analysis["possible_moves"].keys())
    #if not best_moves:
    #    best_moves = list(analysis["possible_moves"].keys())
   # if (best_moves):
    ply=random.choice(spicy_moves)
  #  else:
  #      ply = random.choice(spicy_moves)
    return ply


def very_optimal_k(state: Nim) -> Nimply:
    analysis = analize_k(state)
    
    #logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = None
    count_1=0
    index_row=-1
    row_value=0
    
    
    for index,value in enumerate(state.rows):
     if(value)>1:                                                               
         count_1+=1                                                             
         index_row=index                                                           #   (X,#indiffernte,xor=0)
         row_value=value          # (#numerorighe>1,numerorighe>0,xor)                (1, #dispari,indifferente)
                                                                                      
                               #(0,xor=-1) vincente      (3,-5)  (4,-5) best   
    if(count_1>1):   # (1,xor=0) (2, xor=0) # (1,xor=1)    vincente      valori maggiori di uno
                     
        spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 0]
    
        if not spicy_moves:
            spicy_moves = list(analysis["possible_moves"].keys())
        ply=random.choice(spicy_moves)
        return ply
    if(count_1==1):# only 1 row with more than 1 object    ->   the goals is to keep odd active rows
     
        num_active_rows = sum(o > 0 for o in state.rows)
       # print(num_active_rows)
        #print(state.rows)
       
        if(num_active_rows%2!=0): #if even take all the object in that row
           # print(fitness(state,Nimply(index_row,row_value-1)))
            return Nimply(index_row,min(state._k,row_value-1))
        else:#leave only 1 object in that row
    
          return  Nimply(index_row,min(state._k,row_value))


   
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
     
    if not spicy_moves:
       spicy_moves = list(analysis["possible_moves"].keys())
   
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [115]:
logging.getLogger().setLevel(logging.INFO)
won=0
player=1;
k=2
for m in range (1500):
    strategy = (very_optimal_k,optimal_k)# - > NImsum non è golden rule
    
    nim = Nim(5,2)  #mutazione -> invece di prendere k k-1   xor ->?==0 best  adaptive??  k# <-    parti prendendo al massimo 
    #logging.info(f"init : {nim}")
  
    player = (player + 1) % 2   # switch starting player every match
    while nim:
        #choose strategy early-game -> late-game by looking at nim->  count_1==1  
        

        ply = strategy[player](nim)
        
       # logging.info(f"ply: player {player} plays {ply}")
        #newfitness=fitness(nim,ply)
        nim.nimming(ply)
        #logging.info(f"status: {nim} ->ftiness {newfitness}")
       
       
        player = 1 - player
    if player == 0:
        won += 1
    
   # logging.info(f"status: Player {player} won!")
   #fitness 1 -> 31/50
   #newfittness->
   #early game--lategame     set of rules with weight  -> won   set rules early game - set rules late games  late games=count==1 ok??
  #evolutionari strategi    w1 strategia1 w2 strategia2 w3 strategia3-> voto su weight maggiore sistemo weight-> won/gameplayed and then redo  
                            # (w1,w2,w3,w4)->fitness(won/gameplayed) change random  voto per maggioranza  w1 e w2 soluzione(w1+w2)  w3 piji 
                            #early game una strategia late game altre strattegia
                            #strategie stupide -> prendi dispari prendi solo grandi prendi solo piccoli
                            #go random until = rows>1==1  and then dispari/pari
                            

print(f"Stategy ({strategy[0].__name__}) won {won/1500*100}% games against ({strategy[1].__name__}) strategy")


## Evolutionary 


In [ ]:
@dataclass
class Weights:
    rules: list(tuple[float,Callable])= None



    def add_rule(self, weight, rule_function):
        """Add a rule to the weights."""
        if self.rules is None:
            self.rules = []
        rule = (weight, rule_function)
        self.rules.append(rule)
    
    
    def mutate_weights(self,tweaks):
        """Add a vector to all existing weights."""
      
        if self.rules is not None and len(tweaks) == len(self.rules):
            self.rules = [(weight + tweaks[i], rule_function) for i, (weight, rule_function) in enumerate(self.rules)]
        
    
    

In [ ]:
def choose_py(weights: Weights,state: Nim):
   
    result_sum_dict = {}

    # Iterate over each rule in the Weights object
    for weight, rule_function in weights.rules:
        # Call the rule function with an example Nim instance
        result = rule_function(state)
        
        # Update the sum of weights in the dictionary
        result_sum_dict[result] = result_sum_dict.get(result, 0) + weight

    max_result = max(result_sum_dict, key=result_sum_dict.get) #risultato

   
    
    return max_result
def games(weights,k):
    won=0
    player=1;
    
    enemy_strategy=[optimal_k,optimal_k,optimal_k]
    tot_enemy=len(enemy_strategy)
    tot_games=tot_enemy*5
    for m in range (tot_games):
        strategy = (very_optimal_k,enemy_strategy[tot_games%(int(tot_games/tot_enemy))])# - > NImsum non è golden rule
        nim = Nim(5,k)  
        player = (player + 1) % 2   # switch starting player every match
        while nim:
            if(player==0): #our turn
             
             nim.nimming(choose_py(weights,nim))

            if(player==1): #enemy turn
             ply = strategy[1](nim)
             nim.nimming(ply)
        
        
            player = 1 - player
        if player == 0:
            won += 1
    return won/tot_games*100


def mutate(weight: Weights):
    λ = 20
    o = 0.1
        
    
    tweaks=np.random.normal(loc=0, scale=o, size=(λ,len(weight.rules)))
    offspring=[]
    for tweak in tweaks:
         
         tmp=deepcopy(weight)
         
         tmp.mutate_weights(tweak)
         
         offspring.append(tmp)
         
    return offspring

In [ ]:
w=Weights()
w.add_rule(0.2,optimal_k)
w.add_rule(0.3,pure_random_k)
w.add_rule(0.4,gabriele_k)
solution=w
best_solution=deepcopy(w)
fitness_best=games(best_solution,k)
λ = 2
o = 0.001
k=2
history=list()
for n in tqdm(range(10)):
   
  offspring = (mutate(solution))
  evals=[]
  [evals.append(games(x_offspring,k)) for  x_offspring in  offspring]
  solution=offspring[np.argmax(evals)]
  fitness_solution=games(solution,k)
  if fitness_best < fitness_solution:
    best_solution=deepcopy(solution)
    fitness_best=fitness_solution
    history.append((n, fitness_solution))
  

logging.info(f"Best solution: {fitness_best}")
history = np.array(history)
plt.figure(figsize=(14, 4))
plt.plot(history[:, 0], history[:, 1], marker=".")

 

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html